In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine=pd.read_csv('https://bit.ly/wine_csv_data')

data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier #랜덤 포레스트 알고리즘

rf=RandomForestClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
#return_train_score=True 을 하게 되면 기본적으로 cross_validate에서 나오는 값은 검증세트에 대한 스코어 값이지만 훈련세트의 스코어 값도 반환하여 둘이 비교할 수 있다
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

rf.fit(train_input,train_target)
print(rf.feature_importances_)#특성트리의 중요도
#부트스트랩 샘플을 만들때 남는 샘플을 oob라고 한다. 이것들을 활용해서 마치 검증세트로서 활용 할 수 있음
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)

rf.fit(train_input,train_target)
print(rf.oob_score_)

0.9979796012752471 0.8932079662397274
[0.23853083 0.49468716 0.26678201]
0.8976332499518953


In [3]:
from sklearn.ensemble import ExtraTreesClassifier #엑스트라 트리 알고리즘
#부트스트랩 샘플을 사용하지 않고 노드를 분할 할때 A B C의 특성을 램덤하게 분할하여 불순도가 가장 좋은 샘플을 사용하여 과대적합을 해소
#램덤하게 분할하기 때문에 속도가 굉장히 빠르지만 트리가 많이 생성되면 될수록 성능이 좋아지기 때문에 많은 트리가 필요하다 
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

et.fit(train_input,train_target)
print(et.feature_importances_)

0.9979796012752471 0.8845504183016214
[0.20401879 0.52110469 0.27487653]


In [4]:
from sklearn.ensemble import GradientBoostingClassifier #그레디언트 부스팅
#경사 하강법 과 굉장히 유사한 방법 분류 일때-> 로지스틱 손실함수  회귀 일때->평균 제곱 오차
#손실함수의 값들은 결국 실수 값이 나오는데 실수 값을 낮아지도록 트리를 추가하는 방법 즉 잔여 오차에 대하여 학습하는 트리를 계속하여 추가한다. 
#회귀 트리를 추가 max_depth=3으로 지정-> 깊이가 낮은 트리(과대적합 가능성 적음) but 성능이 좋지 않음 이를 트리를 추가해 성능을 올림
# remind! 손실함수를 정의하고 손실함수의 최저값을 찾는 방법-> 가중치 변경하면서(경사하강법)

gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
#손실함수의 더 낮은 부분으로 이동 경사각도 변경
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

gb.fit(train_input,train_target)
print(gb.feature_importances_)


0.8844523565957292 0.868576108684386
0.9454013426589803 0.8724276301177166
[0.16788068 0.68431409 0.14780523]


In [5]:
from sklearn.experimental import enable_hist_gradient_boosting #히스토그램 기반 그레디언트 부스팅
#특성의 범위가 제한되어 있기 때문에 속도가 빠르고 과대적합을 막고 성능이 좋게 된다.
#255개의 구간으로 나누고 나머지 한개의 구간은 누락된 값에 대해서 할당을 함 

from sklearn.ensemble import HistGradientBoostingClassifier

hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

C:\Users\user\anaconda3\envs\XAI\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9310662092475901 0.8801221588805804


In [6]:
from sklearn.inspection import permutation_importance #permutation importance 치환 중요도
#특성 열과 샘플 행에서 첫번째 특성을 섞음 
#특성의 중요도를 알 수 있음 예를 들어 첫번째 특성을 섞었을때 성능이 80% 두번째 특성을 섞었을 때 성능이 70%이면 
#두번째 성능을 섞었을 때 성능이 더 안좋아졌으므로 두번째 특성이 더욱더 중요한 특성임을 알 수 있음
hgb.fit(train_input,train_target)
result=permutation_importance(hgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
#n_repeats의 기본값은 5

print(result.importances_mean)

result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)

print(result.importances_mean)
hgb.score(test_input,test_target)

C:\Users\user\anaconda3\envs\XAI\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\user\anaconda3\envs\XAI\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[0.09270733 0.23777179 0.08735809]
[0.05484615 0.20123077 0.04538462]


0.8776923076923077

In [7]:
from xgboost import XGBClassifier #XGBoost 알고리즘

xgb=XGBClassifier(tree_method='hist',random_state=42)
#tree_method='hist'로 하면 위의 히스토그램 기반 그레디언트 부스팅 알고리즘으로 사용가능

scores=cross_validate(xgb,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9559361441487976 0.8782005626712076


In [8]:
from lightgbm import LGBMClassifier

lgb=LGBMClassifier(random_state=42)
scores=cross_validate(xgb,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9559361441487976 0.8782005626712076
